In [28]:
import sys
import os
import time
import logging
import datetime
from datetime import datetime, timedelta
import yaml
import importlib
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import requests
import pandas_gbq
from dreams_core.googlecloud import GoogleCloud as dgc
from dreams_core import core as dc
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema

# load dotenv
load_dotenv()

# import local files if necessary
sys.path.append('..//src')
from utils import load_config, cw_filter_df
import training_data as td
importlib.reload(td)
import feature_engineering as fe
importlib.reload(fe)
import coin_wallet_metrics as cwm
importlib.reload(cwm)
import modeling as m
importlib.reload(m)
import insights as i
importlib.reload(i)

# load configs
config = load_config('../config/config.yaml')
metrics_config = load_config('../config/metrics_config.yaml')
modeling_config = load_config('../config/modeling_config.yaml')
experiments_config = load_config('../config/experiments_config.yaml')

# configure logger
logger = dc.setup_logger()
logger.setLevel(logging.INFO)

# Custom format function for displaying numbers
pd.set_option('display.float_format', lambda x: f'{x:.12g}')
# pd.reset_option('display.float_format')

In [29]:
importlib.reload(td)
importlib.reload(cwm)
importlib.reload(fe)
importlib.reload(m)
importlib.reload(i)
config = load_config('../config/config.yaml')
metrics_config = load_config('../config/metrics_config.yaml')
modeling_config = load_config('../config/modeling_config.yaml')
experiments_config = load_config('../config/experiments_config.yaml')


### Training Data Generation

In [3]:
logger.setLevel(logging.INFO)

# retrieve and clean prices data
prices_df = td.retrieve_prices_data()
prices_df,_ = td.fill_prices_gaps(prices_df,config['data_cleaning']['max_gap_days'])

# retrieve transfers data
transfers_df = td.retrieve_transfers_data(
    config['training_data']['training_period_start'],
    config['training_data']['modeling_period_start'],
    config['training_data']['modeling_period_end']
    )

# compile profits_df
profits_df = td.prepare_profits_data(transfers_df, prices_df)
profits_df = td.calculate_wallet_profitability(profits_df)
profits_df,_ = td.clean_profits_df(profits_df, config['data_cleaning'])



[14/Sep/2024 12:25:53] INFO [dreams_core.core.retrieve_prices_data:42] retrieved prices data with shape (120763, 3)
[14/Sep/2024 12:25:53] INFO [dreams_core.core.fill_prices_gaps:126] 382 coins had no gaps, 19 coins had gaps filled, and 38 coins were dropped due to large gaps.
[14/Sep/2024 12:27:08] INFO [dreams_core.core.retrieve_transfers_data:333] retrieved transfers_df with shape (35657183, 5) after 74.9 seconds.
[14/Sep/2024 12:27:08] INFO [dreams_core.core.prepare_profits_data:374] Preparing profits_df data...
[14/Sep/2024 12:28:31] INFO [dreams_core.core.calculate_wallet_profitability:554] Generated profits df after 36.06 seconds
[14/Sep/2024 12:29:09] INFO [dreams_core.core.clean_profits_df:625] Finished cleaning profits_df after 38.51 seconds.


### Metrics and Feature Engineering

In [24]:
importlib.reload(td)
importlib.reload(cwm)
importlib.reload(fe) 
importlib.reload(m) 
config = load_config('../config/config.yaml')
metrics_config = load_config('../config/metrics_config.yaml')
modeling_config = load_config('../config/modeling_config.yaml')



# identify sharks
shark_coins_df = td.classify_shark_coins(profits_df, config['training_data'])
shark_wallets_df = td.classify_shark_wallets(shark_coins_df,config['training_data'])

# generate and flatten buysell_metrics
cohort_wallets = shark_wallets_df[shark_wallets_df['is_shark']==True]['wallet_address'].unique()
cohort_coins = shark_coins_df['coin_id'].unique()
buysell_metrics_df = cwm.generate_buysell_metrics_df(profits_df,config['training_data']['training_period_end'],cohort_wallets,cohort_coins)

# flatten, save, and preprocess the flattened df
output_directory = '..//modeling/outputs/flattened_outputs/'
metric_description = 'buysell_metrics'
modeling_period_start = config['training_data']['modeling_period_start']
version = '0.1'

flattened_buysell_metrics_df = fe.flatten_coin_date_df(buysell_metrics_df,metrics_config,config['training_data']['training_period_end'])
flattened_df, flattened_filepath = fe.save_flattened_outputs(flattened_buysell_metrics_df, output_directory, metric_description, modeling_period_start, version)
preprocessed_df, preprocessed_filepath = fe.preprocess_coin_df(flattened_filepath, modeling_config, metrics_config)

# create the training data df
input_directory = f"{preprocessed_filepath.split('preprocessed_outputs/')[0]}preprocessed_outputs/"
input_filenames = [
    preprocessed_filepath.split('preprocessed_outputs/')[1]
]
training_data_df = fe.create_training_data_df(input_directory, input_filenames)

# create the target variable df
target_variable_df,_ = fe.create_target_variables_mooncrater(prices_df, config['training_data'], modeling_config)

# merge the two into the final model input df
model_input_df = fe.prepare_model_input_df(training_data_df, target_variable_df, modeling_config['modeling']['target_column'])

[14/Sep/2024 12:50:11] INFO [dreams_core.core.classify_shark_coins:691] creation of shark_coins_df complete.
[14/Sep/2024 12:50:11] INFO [dreams_core.core.generate_buysell_metrics_df:33] Preparing buysell_metrics_df...
[14/Sep/2024 12:50:15] INFO [dreams_core.core.generate_buysell_metrics_df:99] Generated buysell_metrics_df after 3.49 seconds.
[14/Sep/2024 12:50:15] INFO [dreams_core.core.flatten_coin_date_df:84] Flattening columns ['buyers_new', 'total_bought', 'total_sold', 'total_buyers'] into coin-level features...
[14/Sep/2024 12:50:15] INFO [dreams_core.core.flatten_coin_date_df:100] Flattened input df into coin-level features with shape (89, 57) after 0.10 seconds.
[14/Sep/2024 12:50:15] WARNING [dreams_core.core.preprocess_coin_df:412] Unknown scaling method None for column total_bought_median
[14/Sep/2024 12:50:15] WARNING [dreams_core.core.preprocess_coin_df:412] Unknown scaling method None for column total_buyers_sum
[14/Sep/2024 12:50:15] INFO [dreams_core.core.preprocess_c

### Modeling

In [30]:
importlib.reload(td)
importlib.reload(cwm)
importlib.reload(fe) 
importlib.reload(m)
config = load_config('../config/config.yaml')
metrics_config = load_config('../config/metrics_config.yaml')
modeling_config = load_config('../config/modeling_config.yaml')
modeling_folder = modeling_config['modeling']['modeling_folder']

model_input_df.shape

# split train/test sets
X_train,X_test,y_train,y_test = m.split_model_input(model_input_df, modeling_config['modeling']['target_column'], test_size=0.2, random_state=42)

# # train model
model, model_id = m.train_model(X_train, y_train, modeling_folder, model_params=None)

# evaluate model
metrics_dict = m.evaluate_model(model, X_test, y_test, model_id, modeling_folder)

# log experiment results
experiment_log = m.log_experiment_results(modeling_folder, model_id)

metrics_dict

[14/Sep/2024 13:44:39] INFO [dreams_core.core.split_model_input:84] y_train: 40/71 positives, y_test: 9/18 positives


{'accuracy': 0.6111111111111112,
 'precision': np.float64(0.5833333333333334),
 'recall': np.float64(0.7777777777777778),
 'f1_score': np.float64(0.6666666666666666),
 'roc_auc': np.float64(0.6419753086419753),
 'log_loss': 0.6697705414644598}

### Experiments

In [43]:
importlib.reload(td)
importlib.reload(cwm)
importlib.reload(fe) 
importlib.reload(m)
importlib.reload(i)
config_folder = '../config'
config = load_config('../config/config.yaml')
metrics_config = load_config('../config/metrics_config.yaml')
modeling_config = load_config('../config/modeling_config.yaml')
modeling_folder = modeling_config['modeling']['modeling_folder']


m.log_experiment_results(modeling_folder, model_id)

{'Model ID': '3a3daba4-51bf-4b06-90f5-0361ac235081',
 'Model parameters': {'n_estimators': 100, 'random_state': 42},
 'accuracy': 0.6111111111111112,
 'precision': 0.5833333333333334,
 'recall': 0.7777777777777778,
 'f1_score': 0.6666666666666666,
 'roc_auc': 0.6419753086419753,
 'log_loss': 0.6697705414644598,
 'feature_importance': '..//modeling/outputs/feature_importance/feature_importance_3a3daba4-51bf-4b06-90f5-0361ac235081.csv'}

In [46]:
importlib.reload(i)


configurations = i.generate_experiment_configurations(config_folder, method='grid', max_evals=50)

FileNotFoundError: config_modeling.yaml not found in ../config

In [ ]:
importlib.reload(td)
importlib.reload(cwm)
importlib.reload(fe) 
importlib.reload(m) 
config = load_config('../config/config.yaml')
metrics_config = load_config('../config/metrics_config.yaml')
modeling_config = load_config('../config/modeling_config.yaml')


# retrieve and clean prices data
prices_df = td.retrieve_prices_data()
prices_df,_ = td.fill_prices_gaps(prices_df,config['data_cleaning']['max_gap_days'])

# retrieve transfers data
transfers_df = td.retrieve_transfers_data(
    config['training_data']['training_period_start'],
    config['training_data']['modeling_period_start'],
    config['training_data']['modeling_period_end']
    )

# compile profits_df
profits_df = td.prepare_profits_data(transfers_df, prices_df)
profits_df = td.calculate_wallet_profitability(profits_df)
profits_df,_ = td.clean_profits_df(profits_df, config['data_cleaning'])

# identify sharks
shark_coins_df = td.classify_shark_coins(profits_df, config['training_data'])
shark_wallets_df = td.classify_shark_wallets(shark_coins_df,config['training_data'])

# generate and flatten buysell_metrics
cohort_wallets = shark_wallets_df[shark_wallets_df['is_shark']==True]['wallet_address'].unique()
cohort_coins = shark_coins_df['coin_id'].unique()
buysell_metrics_df = cwm.generate_buysell_metrics_df(profits_df,config['training_data']['training_period_end'],cohort_wallets,cohort_coins)

# flatten, save, and preprocess the flattened df
output_directory = '..//modeling/outputs/flattened_outputs/'
metric_description = 'buysell_metrics'
modeling_period_start = config['training_data']['modeling_period_start']
version = '0.1'

flattened_buysell_metrics_df = fe.flatten_coin_date_df(buysell_metrics_df,metrics_config,config['training_data']['training_period_end'])
flattened_df, flattened_filepath = fe.save_flattened_outputs(flattened_buysell_metrics_df, output_directory, metric_description, modeling_period_start, version)
preprocessed_df, preprocessed_filepath = fe.preprocess_coin_df(flattened_filepath, modeling_config, metrics_config)

# create the training data df
input_directory = f"{preprocessed_filepath.split('preprocessed_outputs/')[0]}preprocessed_outputs/"
input_filenames = [
    preprocessed_filepath.split('preprocessed_outputs/')[1]
]
training_data_df = fe.create_training_data_df(input_directory, input_filenames)

# create the target variable df
target_variable_df,_ = fe.create_target_variables_mooncrater(prices_df, config['training_data'], modeling_config)

# merge the two into the final model input df
model_input_df = fe.prepare_model_input_df(training_data_df, target_variable_df, modeling_config['modeling']['target_column'])

# split train/test sets
X_train,X_test,y_train,y_test = m.split_model_input(model_input_df, modeling_config['modeling']['target_column'], test_size=0.2, random_state=42)

